In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
import datetime
import gc
import os 

from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import mean_squared_error


import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import lightgbm as lgb

%matplotlib inline
from sklearn.linear_model import (LinearRegression, Ridge, Lasso,LogisticRegression)
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    

    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [ ]:

def missing_statistics(df):    
    statitics = pd.DataFrame(df.isnull().sum()).reset_index()
    statitics.columns=['COLUMN NAME',"MISSING VALUES"]
    statitics['TOTAL ROWS'] = df.shape[0]
    statitics['% MISSING'] = round((statitics['MISSING VALUES']/statitics['TOTAL ROWS'])*100,2)
    return statitics

In [ ]:
training = import_data('/kaggle/input/testval-processing/df_train.csv')
df_val = import_data('/kaggle/input/testval-processing/df_val.csv')


In [ ]:
data_test = import_data('/kaggle/input/testval-processing/data_test.csv')

In [ ]:
gc.collect()


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#training['timestamp'] = pd.to_datetime(training.timestamp, format='%Y-%m-%d %H:%M:%S')
#training['timestamp'] = pd.to_numeric(training['timestamp'])
training.primary_use = le.fit_transform(training.primary_use)
training.set_index(['hour', 'month'], inplace=True)

#data_test['timestamp'] = pd.to_datetime(data_test.timestamp, format='%Y-%m-%d %H:%M:%S')
#data_test['timestamp'] = pd.to_numeric(data_test['timestamp'])
data_test.primary_use = le.fit_transform(data_test.primary_use)
data_test.set_index(['hour', 'month'], inplace=True)
training

In [ ]:
import random

#training = training.sample(frac = 1) 

x_train = training[['site_id','building_id','meter','primary_use','square_feet','air_temperature',
'dew_temperature','wind_direction','wind_speed','sea_level_pressure','precip_depth_1_hr']]
#
y_train = training['meter_reading'] 

X_val = training[['site_id','building_id','meter','primary_use','square_feet','air_temperature',
'dew_temperature','wind_direction','wind_speed','sea_level_pressure','precip_depth_1_hr']]
#
y_val = training['meter_reading'] 

In [ ]:
x_train = x_train.values[:]
x_train= x_train.reshape((x_train.shape[0],1,x_train.shape[-1]))
#x_train = np.expand_dims(X,axis = 2)
y_train = y_train.values

# validation_data=(x_val,y_val)
x_val = X_val.values[:]
x_val = x_val.reshape((x_val.shape[0],1,x_val.shape[-1]))
#x_val = np.expand_dims(X,axis = 2)
y_val= y_val.values
# print(x_train.shape , x_val.shape) 

# ANN Model

In [ ]:
from sklearn import preprocessing
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, LSTM, GRU, Dropout, BatchNormalization 
from keras.layers.core import Activation
from keras.models import Sequential
from keras.optimizers import RMSprop,Adam
from keras import regularizers

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers 
import random
import os

# Import deep learning layers and functions from tensorflow
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam


In [ ]:
# Set all random values of the system
def reset_random_seeds():
   os.environ['PYTHONHASHSEED']=str(1)
   print('tf')
   tf.random.set_seed(1)
   print('tf')
   np.random.seed(1)
   print('np')
   random.seed(1)
   print('np')

In [ ]:
gc.collect()

In [ ]:
#x_train, x_val, y_train, y_val = train_test_split(X,y, test_size = 0.2, random_state= 45 )

In [ ]:

gc.collect()

In [ ]:
checkpoint_path="Best_model/cp-{epoch:04d}.ckpt" 
#checkpoint_path="Best_model/cp.ckpt" # The number of the best epoch is also specified in the ast model path
#-{epoch:04d}
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpoint = tensorflow.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', verbose=1, 
                                                        save_best_only=True, mode='min')


In [ ]:
es = EarlyStopping(monitor='val_root_mean_squared_error', min_delta=0.0001, patience=1, verbose=True, mode='auto')


In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.01) # model optimazer with mearning rat eof 0,01

callbacks_list =[tf.keras.callbacks.EarlyStopping(patience=4,verbose=True, mode='auto'),checkpoint]  # put the checkpoint belong the callbacks_list

# tf.keras.callbacks.TensorBoard(log_dir='./logs') #Write TensorBoard logs after every batch of training to monitor your metrics

my_init = tensorflow.keras.initializers.glorot_uniform(seed=1) # features weights initializer
# Use tensorflow.keras.layers
model = tensorflow.keras.Sequential([
            tensorflow.keras.layers.Dense(200, input_dim=x_train.shape[-1], activation='relu',kernel_initializer=my_init),
            tensorflow.keras.layers.Dense(128, activation='relu',kernel_initializer=my_init),
            tensorflow.keras.layers.Dense(64, activation='relu',kernel_initializer=my_init),
            tensorflow.keras.layers.Dense(1,kernel_initializer=my_init)
        ])

In [ ]:
gc.collect()

In [ ]:
model.compile(optimizer=opt, loss='mean_absolute_error') #Adam(learning_rate=0.001) : this is the default learning rate of adam


In [ ]:
model.summary()

In [ ]:
gc.collect()

In [ ]:
model.fit(x_train, y_train, epochs=100, batch_size=6000, validation_data=(x_val,y_val) ,verbose=1,callbacks=callbacks_list); # include callbacks_list to callbacks parameter


In [ ]:
del training , checkpoint , x_train , y_train

In [ ]:
ls {checkpoint_dir}


In [ ]:
models = os.listdir('Best_model')
models.sort()
m = 'Best_model/'+models[-1]
Best_model = tensorflow.keras.models.load_model(m)
m

In [ ]:
from numpy import sqrt
mse = Best_model.evaluate(x_val, y_val, verbose=0)
print('MSE: %.3f, RMSE: %.3f ' % (mse, sqrt(mse)))

In [ ]:
del x_val , y_val , models , m

In [ ]:
gc.collect()

In [ ]:
test_data = data_test[['site_id','building_id','meter','primary_use','square_feet','air_temperature',
                 'dew_temperature','sea_level_pressure','wind_direction','wind_speed','precip_depth_1_hr']]

test_data = test_data.values[:]

In [ ]:
gc.collect()

In [ ]:
 submission=import_data('/kaggle/input/ashrae-energy-prediction/sample_submission.csv')

In [ ]:
f_predict=Best_model.predict(test_data)

In [ ]:
#len(f_predict[f_predict>1])
np.count_nonzero(~np.isnan(f_predict)) #number of value non Nan


In [ ]:
test = np.concatenate((test_data, f_predict), axis=1)
del f_predict

In [ ]:
submission_pfs = pd.DataFrame(test,columns=['site_id','building_id','meter','primary_use','square_feet','air_temperature',
                 'dew_temperature','sea_level_pressure','wind_direction','wind_speed','precip_depth_1_hr','meter_reading'])

In [ ]:
gc.collect()

In [ ]:
data_to_convtest = submission_pfs[submission_pfs['site_id']==0].index #Convertir les prédictions du site_id  0 

for j in data_to_convtest :
    submission_pfs.at[j,'meter_reading']=(submission_pfs.at[j,'meter_reading']*3.4118)

# submition data

In [ ]:
submission['meter_reading'] = submission_pfs['meter_reading'] 

In [ ]:
del data_to_convtest, submission_pfs

In [ ]:
gc.collect()

In [ ]:
submission['meter_reading'].clip(lower=0,upper=None,inplace=True) 

In [ ]:
submission.to_csv('ANN_sub.csv',index = False)
submission.head(5)